In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import shelve
import time
from sklearn.linear_model import SGDClassifier
start = time.time();
shelve_file = shelve.open('../data/sample')
X_train = shelve_file['X_train']
y_train = shelve_file['y_train']
X_test = shelve_file['X_test']

tfidf = TfidfVectorizer(ngram_range=(1,4), min_df=3, max_df=0.9, use_idf=1, smooth_idf=1, sublinear_tf=1)
tfidf.fit(X_train)
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)
train_start = time.time()
dtc = DecisionTreeClassifier(max_depth=50, min_samples_split=60, min_samples_leaf=70)
sgd = SGDClassifier(loss="modified_huber", alpha=0.00002, n_jobs=-1,penalty = 'l2')
bdt = AdaBoostClassifier(sgd,
                         algorithm="SAMME",
                         n_estimators=100, learning_rate=0.9)
bdt.fit(X_train,y_train)
print('Time comsumption on train',time.time()-train_start)


In [ ]:
test_start = time.time()
prediction = bdt.predict(X_train)
print('Time comsumption on test',time.time()-test_start)
print(metrics.classification_report(y_train,prediction))#分类报告
print(metrics.confusion_matrix(y_train,prediction))#混淆矩阵

In [ ]:
test_start = time.time()
#预测
X_test = tfidf.transform(X_test)
prediction = bdt.predict(X_test)
print('Time comsumption on test',time.time()-test_start)
#保存结果
f_out = open('adaboost.csv', 'w')
f_out.write("id,class"+"\n")
for i in range(X_test.shape[0]):
    f_out.write(str(i)+","+str(prediction[i])+'\n')
f_out.close()
print('Time comsumption:',(time.time()-start)/3600,'h')